# Demo Model

> 

In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F # F.mse_loss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

In [ ]:
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)

In [ ]:
# n_obs = 100 
# n_degree = 5


# n_poly = n_degree + 1
# # X = torch.randn(n_obs).reshape((n_obs, 1))
# X = torch.linspace(-10, 10, n_obs).reshape((n_obs, 1))
# coefs_poly = torch.randn((n_poly))
# # coefs_poly = coefs_poly/torch.linspace(1, n_poly, n_poly)
# y = (coefs_poly * (X**torch.tensor([i for i in range(n_poly)]))).sum(axis = 1).reshape((-1, 1))

In [ ]:
sim_obs = 1001
bounds = (-4, 4)
X = torch.linspace(bounds[0], bounds[1], sim_obs).reshape((sim_obs, 1))[:, 0]
y = torch.Tensor(
    0.01*(
        X**6 - 
      2*X**5 - 
     26*X**4 + 
     28*X**3 + 
    145*X**2 - 
     26*X    - 
     80)
     )

In [ ]:
class ds(Dataset):
    def __init__(self, y, X): 
        self.y = y
        self.X = X
    def __len__(self): 
        return(len(self.y))
    def __getitem__(self, index):
        return self.y[index], self.X[index]

In [ ]:
px.scatter(x = torch.Tensor(X).numpy(), 
           y = torch.Tensor(y).numpy())

In [ ]:
tmp=pd.DataFrame({
    'F(x)':0.5*X**2,
    'x':X})
tmp['x1.Linear']= [e for e in tmp['x']]
tmp['x1.NonLinear']= [e if e >0 else 0 for e in tmp['x1.Linear']]

tmp['x2.Linear']= [-2*e for e in tmp['x']]
tmp['x2.NonLinear']= [e if e >0 else 0 for e in tmp['x2.Linear']]

tmp['yhat.Linear']= tmp['x1.Linear']+tmp['x2.Linear']
tmp['yhat.NonLinear']= tmp['x1.NonLinear']+tmp['x2.NonLinear']
px.scatter(tmp.melt('x', value_name='output'), x='x', y='output', facet_col='variable')


In [ ]:
n_obs = 50

even_idxs = torch.linspace(0, X.shape[0]-1, n_obs).int()

clustered_idxs = (torch.randn(n_obs)*100) + sim_obs//2 # center b/c these will become indices
clustered_idxs = clustered_idxs.int()
# clamp to min /max
clustered_idxs[(clustered_idxs < 0)] = 0
clustered_idxs[(clustered_idxs >= sim_obs)] = sim_obs-1

# px.histogram(x = clustered_idxs, nbins=100)

In [ ]:
# all data
df = pd.DataFrame(torch.concat([torch.Tensor(X)[:, None], torch.Tensor(y)[:, None]], axis = 1).numpy(), columns=['x', 'y'])


In [ ]:
px.scatter(
    pd.concat([
        df.assign(Sample = 'All'),
        df.loc[even_idxs].assign(Sample = 'Unbiased'),
        df.loc[clustered_idxs].assign(Sample = 'Biased')
        ]),
    x = 'x', y='y', color = 'Sample'
)

In [ ]:
# y[even_idxs, None]

# even_idxs = [int(e) for e in even_idxs]

# X[even_idxs, None]

# even_idxs = [int(e) for e in range(50)]


In [ ]:
batch_size = 25

training_dataloader_even = DataLoader(
    ds(y = y[even_idxs, None], 
       X = X[even_idxs, None]),
    batch_size = batch_size,
    shuffle = True
)


training_dataloader_uneven = DataLoader(
    ds(y = y[clustered_idxs, None], 
       X = X[clustered_idxs, None]),
    batch_size = batch_size,
    shuffle = True
)



valid_dataloader = DataLoader(
    ds(y = y[:, None], 
       X = X[:, None]),
    batch_size = batch_size,
    shuffle = False
)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, inp_size = 1, out_size = 1, hidden_layers = []):
        super(NeuralNetwork, self).__init__()    

        hidden_layers = [inp_size]+hidden_layers+[out_size]   
        print(hidden_layers)     
        layer_list = []
        for i in range(len(hidden_layers)):
            if i+2 > len(hidden_layers):
                pass
            elif i+2 < len(hidden_layers):
                layer_list += [nn.Linear(hidden_layers[i], hidden_layers[i+1]), 
                               nn.ReLU()]
            else:
                layer_list += [nn.Linear(hidden_layers[i], hidden_layers[i+1])]
        self.x_network = nn.ModuleList(layer_list)

    def forward(self, x):
        for m in self.x_network:
            x = m(x)
        return x
    
model = NeuralNetwork(inp_size = 1, out_size = 1, hidden_layers = [])
# model( next(iter(training_dataloader_even))[1][:, None] )[0:3]

In [ ]:
from tqdm import tqdm
import pandas as pd


class ModelHelper():
    def __init__(self, model, ds_train, ds_valid) -> None:
        self.ds_train = ds_train 
        self.ds_valid = ds_valid

        self.model = model
        # self.optimizer = torch.optim.Adam(model.parameters(), lr= 1e-3)        
        self.optimizer = torch.optim.Adam(model.parameters(), lr= 1e-2)        
        self.loss_fn = F.mse_loss
    
        self.epoch = 0
        self.history = {
            'train': {'epoch':[], 'loss':[]},
            'valid': {'epoch':[], 'loss':[]}
        }

        self.yhats = {
            'valid': {'epoch':[], 'yhat':[]}
        }

    def _epoch_optim(self, dataloader):
        for batch, (y_i, xs_i) in enumerate(dataloader):
            pred = self.model(xs_i)
            loss = self.loss_fn(pred, y_i)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

    def _calc_ds_loss(self, dataloader):
        with torch.no_grad():
            loss_tally = []
            obs_tally = []
            for batch, (y_i, xs_i) in enumerate(dataloader):
                pred = self.model(xs_i)
                loss = self.loss_fn(pred, y_i)   
                loss_tally+= [loss[None]]
                obs_tally += [len(y_i)]

            obs_tally = torch.tensor(obs_tally)
            # calculate average of batch mse weighted by batch size (in case of inequal batch sizes)
            return float(sum(torch.concat(loss_tally, axis = 0)*(obs_tally/sum(obs_tally))))
    
    def _calc_yhats(self, dataloader):
        with torch.no_grad():
            yhats = []
            for batch, (y_i, xs_i) in enumerate(dataloader):
                yhats += [self.model(xs_i)]
            return torch.concat(yhats)

    def log_loss(self):
        self.history['train']['epoch'] += [self.epoch]
        self.history['valid']['epoch'] += [self.epoch]
        self.history['train']['loss']  += [self._calc_ds_loss(dataloader=self.ds_train)]
        self.history['valid']['loss']  += [self._calc_ds_loss(dataloader=self.ds_valid)]

    def log_yhats(self):
        self.yhats['valid']['epoch'] += [self.epoch]
        self.yhats['valid']['yhat']  += [self._calc_yhats(dataloader=self.ds_valid)]

    def train_epoch(self):
        self.epoch += 1
        self._epoch_optim(dataloader = self.ds_train)
        

    def train(self, epochs = 1, save_yhat_every = 1):
        self.log_yhats()
        for i in tqdm(range(epochs)):
            self.train_epoch()
            self.log_loss()
            if self.epoch % save_yhat_every == 0:
                self.log_yhats()

    def tidy_history(self):
        history = pd.concat([
            pd.DataFrame(self.history['train']).assign(split = 'train'),
            pd.DataFrame(self.history['valid']).assign(split = 'valid')])
        return history

    def tidy_history_yhats(self):        
        res_list = []
        
        epochs_run = len(self.yhats['valid']['epoch'])
        for i in range(epochs_run):
            vals = self.yhats['valid']['yhat'][i]
            res_list += [torch.concat([
                vals, 
                torch.tensor(
                    self.yhats['valid']['epoch'][i]
                    ).repeat(vals.shape[0])[:, None]], 
                    axis = 1
            )]
        yhat_df = pd.DataFrame(torch.concat(res_list).numpy(), columns=['yhat', 'epoch'])
        x_df = pd.DataFrame(torch.concat([xs_i for batch, (_, xs_i) in enumerate(self.ds_valid)]).repeat([epochs_run, 1]).numpy(), columns = ['x'])
        yhat_df = pd.concat([yhat_df, x_df], axis = 1)
        yhat_df.epoch = yhat_df.epoch.astype(int)
        return yhat_df


In [ ]:
mh = ModelHelper(model = NeuralNetwork(),
                 ds_train = training_dataloader_even, 
                 ds_valid = valid_dataloader)

mh.log_loss()
mh.train(epochs=100, save_yhat_every = 10)

In [ ]:
px.line(mh.tidy_history(), x = 'epoch', y = 'loss', color = 'split')
# mh.tidy_history_yhats()

In [ ]:
tmp = mh.tidy_history_yhats()

def _add_ref_to_yhats(tmp, y, X):
    ref = pd.DataFrame(
        torch.concat([
            torch.concat([
                torch.Tensor(y)[:, None], 
                torch.Tensor(X)[:, None]], 
                axis = 1).repeat([len(list(set(tmp.epoch))), 1]), 
                
                torch.tensor(tmp.epoch)[:, None]
                ], axis = 1
        ).numpy(), 
                columns=['yhat', 'x', 'epoch'])

    ref.epoch = ref.epoch.astype(int)
    ref['type'] = 'F(x)'
    tmp['type'] = 'Predicted'
    tmp = pd.concat([ref.loc[(ref.epoch.isin(list(set(tmp.epoch)))), ], tmp], axis = 0 )
    return(tmp)

# fig = px.scatter(_add_ref_to_yhats(mh.tidy_history_yhats(), y = y, X= X), x = 'x', y = 'yhat', color = 'type', animation_frame='epoch')
# # # https://community.plotly.com/t/how-to-slow-down-animation-in-plotly-express/31309/5
# fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 0.1
# fig.show()

In [ ]:
nnet = NeuralNetwork(inp_size = 1, out_size = 1, hidden_layers = [])
mh = ModelHelper(model = nnet,
                 ds_train = training_dataloader_even, 
                 ds_valid = valid_dataloader)

mh.log_loss()
mh.train(epochs=100, save_yhat_every = 5)

fig = px.scatter(_add_ref_to_yhats(mh.tidy_history_yhats(), y = y, X= X), x = 'x', y = 'yhat', color = 'type', animation_frame='epoch', title='Linear Model')
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1
fig.show()

In [ ]:
nnet = NeuralNetwork(inp_size = 1, out_size = 1, hidden_layers = [1])
mh = ModelHelper(model = nnet,
                 ds_train = training_dataloader_even, 
                 ds_valid = valid_dataloader)

mh.log_loss()
mh.train(epochs=500, save_yhat_every = 10)

fig = px.scatter(_add_ref_to_yhats(mh.tidy_history_yhats(), y = y, X= X), x = 'x', y = 'yhat', color = 'type', animation_frame='epoch', title='Hidden Units: 1')
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1
fig.show()


In [ ]:
nnet = NeuralNetwork(inp_size = 1, out_size = 1, hidden_layers = [16])
mh = ModelHelper(model = nnet,
                 ds_train = training_dataloader_even, 
                 ds_valid = valid_dataloader)

mh.log_loss()
mh.train(epochs=500, save_yhat_every = 10)

fig = px.scatter(_add_ref_to_yhats(mh.tidy_history_yhats(), y = y, X= X), x = 'x', y = 'yhat', color = 'type', animation_frame='epoch', title='Hidden Units: 16')
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1
fig.show()


In [ ]:
nnet = NeuralNetwork(inp_size = 1, out_size = 1, hidden_layers = [16,16])
mh = ModelHelper(model = nnet,
                 ds_train = training_dataloader_even, 
                 ds_valid = valid_dataloader)

mh.log_loss()
mh.train(epochs=500, save_yhat_every = 10)

fig = px.scatter(_add_ref_to_yhats(mh.tidy_history_yhats(), y = y, X= X), x = 'x', y = 'yhat', color = 'type', animation_frame='epoch', title='Hidden Units: 16, 16')
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1
fig.show()

In [ ]:
# What happens as we increase capacity? Depth?
n_layers = 1
n_units =1

temp_list = []

for n_layers in [i for i in range(1, 2)]:
    for n_units in [i for i in [1]+[10*e for e in range(1, 21)]]:
        nnet = NeuralNetwork(inp_size = 1, out_size = 1, hidden_layers = [n_units for i in range(n_layers)])
        mh = ModelHelper(model = nnet,
                        ds_train = training_dataloader_even, 
                        ds_valid = valid_dataloader)

        mh.log_loss()
        mh.train(epochs=500, save_yhat_every = 500)
        temp = _add_ref_to_yhats(mh.tidy_history_yhats(), y = y, X= X)

        temp['layers']=n_layers
        temp['units']=n_units

        temp_list += [temp]

In [ ]:
tmp=pd.concat(temp_list)
tmp=tmp.loc[tmp.epoch==500,]

fig = px.scatter(tmp, x = 'x', y = 'yhat', color = 'type', animation_frame='units', title='Increasing Units 1 -> 200')
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 4
fig.show()

In [ ]:
temp_list = []

for n_layers in [i for i in range(1, 21, 1)]:
    for n_units in [i for i in [10]]:
        nnet = NeuralNetwork(inp_size = 1, out_size = 1, hidden_layers = [n_units for i in range(n_layers)])
        mh = ModelHelper(model = nnet,
                        ds_train = training_dataloader_even, 
                        ds_valid = valid_dataloader)

        mh.log_loss()
        mh.train(epochs=500, save_yhat_every = 500)
        temp = _add_ref_to_yhats(mh.tidy_history_yhats(), y = y, X= X)

        temp['layers']=n_layers
        temp['units']=n_units

        temp_list += [temp]

In [ ]:
tmp=pd.concat(temp_list)
tmp=tmp.loc[tmp.epoch==500,]

fig = px.scatter(tmp, x = 'x', y = 'yhat', color = 'type', animation_frame='layers', title='Increasing layers 1 -> 20')
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 250
fig.show()

In [ ]:
px.scatter(tmp, x = 'x', y = 'yhat', color = 'type', facet_col='layers', title='Increasing layers 1 -> 20')

In [ ]:
nnet = NeuralNetwork(inp_size = 1, out_size = 1, hidden_layers = [256, 256, 256, 256])
mh = ModelHelper(model = nnet,
                 ds_train = training_dataloader_even, 
                 ds_valid = valid_dataloader)

mh.log_loss()
mh.train(epochs=500, save_yhat_every = 25)

fig = px.scatter(_add_ref_to_yhats(mh.tidy_history_yhats(), y = y, X= X), x = 'x', y = 'yhat', color = 'type', animation_frame='epoch', title='Unbiased Sampling')
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1
fig.show()

In [ ]:
nnet = NeuralNetwork(inp_size = 1, out_size = 1, hidden_layers = [256, 256, 256, 256])
mh = ModelHelper(model = nnet,
                 ds_train = training_dataloader_uneven, 
                 ds_valid = valid_dataloader)

mh.log_loss()
mh.train(epochs=500, save_yhat_every = 25)

fig = px.scatter(_add_ref_to_yhats(mh.tidy_history_yhats(), y = y, X= X), x = 'x', y = 'yhat', color = 'type', animation_frame='epoch', title='Biased Sampling')
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1
fig.show()

In [ ]:
px.line(mh.tidy_history(), x = 'epoch', y = 'loss', color = 'split')

In [ ]:
nnet = NeuralNetwork(inp_size = 1, out_size = 1, hidden_layers = [])
mh = ModelHelper(model = nnet,
                 ds_train = training_dataloader_uneven, 
                 ds_valid = valid_dataloader)

mh.log_loss()
mh.train(epochs=500, save_yhat_every = 25)

fig = px.scatter(_add_ref_to_yhats(mh.tidy_history_yhats(), y = y, X= X), x = 'x', y = 'yhat', color = 'type', animation_frame='epoch', title='Biased Sampling')
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1
fig.show()

In [ ]:
px.line(mh.tidy_history(), x = 'epoch', y = 'loss', color = 'split')